In [ ]:
# pip install selenium
# pip install webdriver-manager


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time

options = Options()
options.add_argument("--start-maximized")  # 창 크게
# options.add_argument("--headless")  # 필요하면 나중에

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

url = "https://korean.visitkorea.or.kr/detail/cs_detail_cos.do?cotid=d7eb914e-8815-4717-8752-bdc154995944&big_category=C01&mid_category=C0115&big_area=1"

driver.get(url)

time.sleep(5)  # 일단 단순 대기



# print("페이지 열림 확인")


title = driver.find_element(By.CSS_SELECTOR, ".tit h2").text
print("코스 제목:", title)

# html = driver.page_source
# print(len(html))

# slides = driver.find_elements(By.CSS_SELECTOR, "li.swiper-slide")

# for slide in slides:
#     print(slide.text)


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time

options = Options()
options.add_argument("--start-maximized")  # 창 크게
# options.add_argument("--headless")  # 필요하면 나중에

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

url = "https://korean.visitkorea.or.kr/detail/cs_detail_cos.do?cotid=d7eb914e-8815-4717-8752-bdc154995944&big_category=C01&mid_category=C0115&big_area=1"

driver.get(url)

time.sleep(5)  # 일단 단순 대기

title = driver.find_element(By.CSS_SELECTOR, ".tit h2").text
area = driver.find_element(By.CSS_SELECTOR, "span.address").text

print({"코스 제목:", title},{"지역:",area})



# slides = driver.find_elements(By.CSS_SELECTOR, "li.swiper-slide")

# slide_data = []

# for slide in slides:
#     course_no = slide.find_element(By.TAG_NAME, "em").text
#     title = slide.find_element(By.CSS_SELECTOR, ".tit span").text
#     href = slide.find_element(By.TAG_NAME, "a").get_attribute("href")
    
#     # 이미지 url
#     style = slide.find_element(By.CSS_SELECTOR, ".img").get_attribute("style")
    
#     import re
#     img_url = re.search(r'url\((.*?)\)', style).group(1)

#     slide_data.append({
#         "course_no": course_no,
#         "title": title,
#         "href": href,
#         "img_url": img_url
#     })

# print(slide_data)


{'코스 제목:', '5코스서울의 대문 숭례문과 그 주변의 사람들을 찾아'} {'서울 중구', '지역:'} {'태그:', ''}


In [25]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
from urllib.parse import urlparse, parse_qs



options = Options()
options.add_argument("--start-maximized")  # 창 크게

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

url = "https://korean.visitkorea.or.kr/detail/cs_detail_cos.do?cotid=d7eb914e-8815-4717-8752-bdc154995944&big_category=C01&mid_category=C0115&big_area=1"

driver.get(url)

parsed = urlparse(driver.current_url)
cotid = parse_qs(parsed.query).get("cotid", [""])[0]

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "detail_box"))
)


title = driver.find_element(By.CSS_SELECTOR, ".tit h2").text
area = driver.find_element(By.CSS_SELECTOR, "span.address").text

desc = driver.find_element(By.CSS_SELECTOR, ".inr_wrap p")
description = driver.execute_script(
    "return arguments[0].innerText;",
    desc
).strip()


schedule_data = {}

schedule_info = driver.find_element(By.CLASS_NAME, "schedule_info")

# 일정, 테마
items = schedule_info.find_elements(By.CSS_SELECTOR, "ul.schedule li")

for item in items:
    key = item.find_element(By.TAG_NAME, "em").text.strip()
    value = item.find_element(By.TAG_NAME, "strong").text.strip()

    schedule_data[key] = value

# 태그
tags = schedule_info.find_elements(By.CSS_SELECTOR, "div.tag span a")

schedule_data["태그"] = [
    driver.execute_script("return arguments[0].innerText;", tag).strip()
    for tag in tags
]

results = []

cos_boxes = driver.find_elements(By.CLASS_NAME, "detail_box")

for box in cos_boxes:
    course_data = {}

    course_data["cos_no"] = driver.execute_script(
        "return arguments[0].innerText;",
        box.find_element(By.TAG_NAME, "em")
    ).strip()

    course_data["cos_title"] = driver.execute_script(
        "return arguments[0].innerText;",
        box.find_element(By.CSS_SELECTOR, "strong a")
    ).strip()

    course_data["cos_add"] = driver.execute_script(
        "return arguments[0].innerText;",
        box.find_elements(By.TAG_NAME, "span")[0]
    ).strip()

    tags = box.find_elements(By.CSS_SELECTOR, "ul.tag li a")
    course_data["cos_tag"] = [
        driver.execute_script("return arguments[0].innerText;", t).strip()
        for t in tags
    ]

    results.append(course_data)

page_data = {
    "cotid": cotid,
    "코스 이름": title,
    "코스 지역" : area,
    "코스 개요": description,
    "일정": schedule_data.get("일정"),
    "테마": schedule_data.get("테마"),
    "페이지태그": schedule_data.get("태그"),
    "courses": results  # 아까 만든 코스 리스트
}

all_pages = []
all_pages.append(page_data)

print(all_pages)

[{'cotid': 'd7eb914e-8815-4717-8752-bdc154995944', '코스 이름': '5코스서울의 대문 숭례문과 그 주변의 사람들을 찾아', '코스 지역': '서울 중구', '코스 개요': '서울을 상진이라 할 수 있는 숭례문과 주변의 관광명소들을 걷는 코스다. 1891년도에 건립된 약현성당은 주변에 약초밭이 있어서 약현이라 불리게 되었단다. 무엇보다 어느 계절에 걸어도 아름다운 정동길과 덕수궁 걷는 길은 아직도 예전의 운치를 그대로 느낄 수 있는 길이다.\n\n출처:이야기를 따라 한양 도성을 걷다.', '일정': '당일여행', '테마': '도보코스', '페이지태그': ['#1박2일', '#2박3일', '#당일코스', '#추천코스'], 'courses': [{'cos_no': '1', 'cos_title': '중명전', 'cos_add': '서울특별시 중구 정동길 41-11 (정동)', 'cos_tag': ['#가족여행', '#고종황제편전', '#관광지', '#교과서속여행', '#근대문화유산', '#대한제국궁궐', '#대한제국편전', '#도심여행', '#상설전시', '#아이와함께', '#역사', '#외국인사교클럽', '#월간_사진_제보_이벤트', '#유명건물', '#을사늑약장소', '#전통&역사문화체험', '#중명전', '#체험학습', '#친구와함께', '#헤이그특사파견', '#황실도서관']}, {'cos_no': '2', 'cos_title': '서소문역사공원', 'cos_add': '서울특별시 중구 칠패로 5 (의주로2가)', 'cos_tag': ['#관광지', '#도심산책', '#산책하기좋은곳', '#서소문성지', '#아이와함께', '#역사공부', '#역사이야기', '#휴식공간', '#휴식여행', '#휴식하기좋은곳', '#힐링산책']}, {'cos_no': '3', 'cos_title': '숭례문', 'cos_add': '서울특별시 중구 세종대로 40', 'cos_tag': ['#관광지', '#교과서속여행', '#국보', '#국보1

In [ ]:
import json
import os
import pandas as pd

SAVE_DIR = "data"
JSON_PATH = os.path.join(SAVE_DIR, "visitkorea_courses.json")
CSV_PATH = os.path.join(SAVE_DIR, "visitkorea_courses.csv")


def save_page_data(new_page_data):

    os.makedirs(SAVE_DIR, exist_ok=True)

    # -------------------------
    # 1️⃣ 기존 JSON 불러오기
    # -------------------------
    if os.path.exists(JSON_PATH):
        with open(JSON_PATH, "r", encoding="utf-8") as f:
            try:
                all_pages = json.load(f)
            except json.JSONDecodeError:
                all_pages = []
    else:
        all_pages = []

    # -------------------------
    # 2️⃣ 동일 cotid 제거 (있으면 삭제 → 덮어쓰기 개념)
    # -------------------------
    all_pages = [
        page for page in all_pages
        if page.get("cotid") != new_page_data["cotid"]
    ]

    # -------------------------
    # 3️⃣ 새 데이터 추가
    # -------------------------
    all_pages.append(new_page_data)

    # -------------------------
    # 4️⃣ JSON 저장
    # -------------------------
    with open(JSON_PATH, "w", encoding="utf-8") as f:
        json.dump(all_pages, f, ensure_ascii=False, indent=4)

    print("✅ JSON 저장 완료")

    # -------------------------
    # 5️⃣ CSV 전체 재생성 (cotid 기준 완전 정리)
    # -------------------------
    flat_rows = []

    for page in all_pages:
        for course in page["courses"]:
            row = {
                "cotid": page["cotid"],
                "코스 이름": page["코스 이름"],
                "코스 지역": page["코스 지역"],
                "코스 개요": page["코스 개요"],
                "일정": page.get("일정"),
                "테마": page.get("테마"),
                "페이지태그": ", ".join(page.get("페이지태그", [])),
                "세부코스 번호": course["cos_no"],
                "세부코스 제목": course["cos_title"],
                "세부코스 주소": course["cos_add"],
                "세부코스 태그": ", ".join(course["cos_tag"])
            }
            flat_rows.append(row)

    df = pd.DataFrame(flat_rows)
    df.to_csv(CSV_PATH, index=False, encoding="utf-8-sig")

    print("✅ CSV 재생성 완료 (cotid 기준 정리)")


In [27]:
save_page_data(page_data)

✅ JSON 저장 완료
✅ CSV 재생성 완료 (cotid 기준 정리)
